In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
import numpy as np

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## Создаём Баесовскую сеть

In [2]:
#Создаём баесовскую сеть
bayesNet = BayesianNetwork()

#Создаём вершины (B - burglary, e - earthqk, A - alarm, J - John calls, M - Mary calls
bayesNet.add_node("B")
bayesNet.add_node("E")
bayesNet.add_node("A")
bayesNet.add_node("J")
bayesNet.add_node("M")

In [3]:
#Создаём грани
bayesNet.add_edge("B", "A")
bayesNet.add_edge("E", "A")
bayesNet.add_edge("A", "J")
bayesNet.add_edge("A", "M")

## Записываем таблицы вероятностей для вершин сети

In [4]:
cpd_B = TabularCPD('B', 2, values=[[.999], [.001]])
cpd_E = TabularCPD('E', 2, values=[[.998], [.002]])

cpd_A = TabularCPD('A', 2, values=[[0.999, .71, .06, .05],
                                   [.001, .29, .94, .95]],
                   evidence=['B', 'E'], evidence_card=[2, 2])

cpd_J = TabularCPD('J', 2,
                   values=[[0.95, .1],
                           [.05, .9]],
                   evidence=['A'], evidence_card=[2])

cpd_M = TabularCPD('M', 2,
                   values=[[0.99, .3],
                           [.01, .7]],
                   evidence=['A'], evidence_card=[2])
bayesNet.add_cpds(cpd_B, cpd_E, cpd_A, cpd_J, cpd_M)

## Проверяем модель на корректность работы

In [5]:
bayesNet.check_model()
print("Model is correct.")

Model is correct.


## Созданём solver, который использует внутреннее исключение переменных для логического вывода

In [6]:
solver = VariableElimination(bayesNet)

# Логический вывод для A и J|E=1

In [7]:
result = solver.query(variables=['A'])
print("A", result)

A +------+----------+
| A    |   phi(A) |
+======+==========+
| A(0) |   0.9975 |
+------+----------+
| A(1) |   0.0025 |
+------+----------+


In [8]:
result = solver.query(variables=['J'], evidence={'E': 1})
print("J|E=1", result)

J|E=1 +------+----------+
| J    |   phi(J) |
+======+==========+
| J(0) |   0.7029 |
+------+----------+
| J(1) |   0.2971 |
+------+----------+
